In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
from sklearn.preprocessing import LabelEncoder
import os


como no hay forma de que funcione la función recomendación, trabajo con una muestra calculada con la calculadoras de https://www.questionpro.com/es/calculadora-de-muestra.html

![calculadora](../Recomendacion_peliculas/assets/calculo%20de%20muestra.jpg)

In [26]:
def recomendacion(titulo: str):
    # Ruta relativa al archivo
    movies_path = os.path.join('Datasets', 'muestra_movies.csv')
    combined_features_path = os.path.join('Datasets', 'muestra_features_optimized.npz')

    # Cargar las bases de datos
    movies = pd.read_csv(movies_path)
    combined_features = np.load(combined_features_path)['arr_0']
    
    # Encuentra el índice de la película en el DataFrame
    try:
        idx = movies[movies['title'].str.lower() == titulo.lower()].index[0]
    except IndexError:
        return [{"error": f"La película '{titulo}' no se encuentra en el dataset."}]

    # Calcular la similitud de coseno entre la película seleccionada y todas las demás
    similitudes = cosine_similarity(combined_features[idx].reshape(1, -1), combined_features)

    # Ordenar las películas por similitud (exceptuando la película actual)
    indices_similares = similitudes[0].argsort()[::-1][1:6]  # Toma las 5 más similares, excluyendo la misma

    # Generar la lista de recomendaciones
    recomendaciones = [
        {
            "titulo": movies.iloc[i]['title'],
            "similitud": float(similitudes[0][i])
        }
        for i in indices_similares
    ]

    return recomendaciones


In [13]:
recomendacion('Among Friends')

ValueError: Input contains NaN.

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.decomposition import PCA
import ast

# Leer el archivo original y seleccionar las columnas necesarias
movies = pd.read_csv(r'Datasets/transformed_movies.csv')
movies_reduced = movies[['title', 'popularity', 'collection', 'genres_encoded']]

# Rellenar valores nulos en la columna de colecciones
movies_reduced['collection'] = movies_reduced['collection'].fillna('No Collection')

# Asegurarse de que los géneros estén en formato de lista
def process_genres(genre_str):
    try:
        return ast.literal_eval(genre_str)
    except ValueError:
        return []

movies_reduced['genres_encoded'] = movies_reduced['genres_encoded'].apply(process_genres)

# Procesar géneros usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(movies_reduced['genres_encoded'])
movies_features = pd.DataFrame(genres_matrix, columns=mlb.classes_)

# Procesar colecciones como una sola variable escalada
collection_counts = movies_reduced['collection'].value_counts()
movies_reduced['collection_weighted'] = movies_reduced['collection'].map(collection_counts).fillna(0)
scaler = StandardScaler()
movies_reduced['collection_weighted_scaled'] = scaler.fit_transform(movies_reduced[['collection_weighted']])

# Combinar características: géneros y colecciones escaladas
movies_features['collection_scaled'] = movies_reduced['collection_weighted_scaled'].values

# Agregar popularidad como una característica más
movies_features['popularity'] = movies_reduced['popularity'].values

# Verificar dimensiones finales
print("Dimensiones finales de las características:", movies_features.shape)

# Guardar la matriz combinada en un archivo comprimido
np.savez_compressed("Datasets/muestra_combined_features.npz", movies_features.values)

# Guardar el DataFrame reducido como CSV
movies_reduced[['title', 'popularity', 'collection', 'genres_encoded']].to_csv(r'Datasets/movies_reduced.csv', index=False)

print("Archivos generados exitosamente.")

C:\Users\paula\AppData\Local\Temp\ipykernel_2448\2677896287.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reduced['collection'] = movies_reduced['collection'].fillna('No Collection')
C:\Users\paula\AppData\Local\Temp\ipykernel_2448\2677896287.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reduced['genres_encoded'] = movies_reduced['genres_encoded'].apply(process_genres)
C:\Users\paula\AppData\Local\Temp\ipykernel_2448\2677896287.py:30: SettingWithCopyWarning: 
A value is trying to

Dimensiones finales de las características: (45376, 22)
Archivos generados exitosamente.


In [11]:
# Crear una copia del DataFrame para evitar el SettingWithCopyWarning
movies_reduced = movies[['title', 'popularity', 'collection', 'genres_encoded']].copy()

# Rellenar valores nulos en la columna de colecciones
movies_reduced['collection'] = movies_reduced['collection'].fillna('No Collection')

# Asegurarse de que los géneros estén en formato de lista
def process_genres(genre_str):
    try:
        return ast.literal_eval(genre_str)
    except ValueError:
        return []

movies_reduced['genres_encoded'] = movies_reduced['genres_encoded'].apply(process_genres)

# Procesar géneros usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(movies_reduced['genres_encoded'])
movies_features = pd.DataFrame(genres_matrix, columns=mlb.classes_)

# Procesar colecciones como una sola variable escalada
collection_counts = movies_reduced['collection'].value_counts()
movies_reduced['collection_weighted'] = movies_reduced['collection'].map(collection_counts).fillna(0)

scaler = StandardScaler()
movies_reduced['collection_weighted_scaled'] = scaler.fit_transform(movies_reduced[['collection_weighted']])

# Combinar características: géneros y colecciones escaladas
movies_features['collection_scaled'] = movies_reduced['collection_weighted_scaled'].values

# Agregar popularidad como una característica más
movies_features['popularity'] = movies_reduced['popularity'].values

# Verificar dimensiones finales
print("Dimensiones finales de las características:", movies_features.shape)

# Guardar la matriz combinada en un archivo comprimido
np.savez_compressed("Datasets/muestra_combined_features.npz", movies_features.values)

# Guardar el DataFrame reducido como CSV
movies_reduced[['title', 'popularity', 'collection', 'genres_encoded']].to_csv(r'Datasets/movies_reduced.csv', index=False)

print("Archivos generados exitosamente.")



Dimensiones finales de las características: (45376, 22)
Archivos generados exitosamente.


In [26]:
print("Dimensiones de collection_encoded:", collection_encoded.shape)


Dimensiones de collection_encoded: (45000, 1691)


In [27]:
print("Número de valores únicos en 'collection':", muestra_movies['collection'].nunique())
print("Primeros 10 valores únicos:\n", muestra_movies['collection'].unique())


Número de valores únicos en 'collection': 1691
Primeros 10 valores únicos:
 ['No Collection' 'Friday the 13th Collection' 'American Pie Collection'
 ... 'Journey to the West Collection' 'Inuyasha Collection'
 'Miles Jupp Standup Collection']


In [9]:
# Convertir todos los nombres de las columnas a cadenas
movies_features.columns = movies_features.columns.astype(str)

# Escalar todas las características
scaler = StandardScaler()
movies_features_scaled = scaler.fit_transform(movies_features)

# Verificar dimensiones finales
print("Dimensiones finales de las características escaladas:", movies_features_scaled.shape)

# Guardar la matriz combinada
np.savez_compressed("Datasets/muestra_combined_features.npz", movies_features_scaled)

print("Archivo comprimido guardado exitosamente.")


Dimensiones finales de las características escaladas: (12186, 905)
Archivo comprimido guardado exitosamente.


In [13]:
import os

In [16]:
def recomendacion(titulo: str):
    # Ruta relativa al archivo
    movies_path = os.path.join('Datasets', 'movies_reduced.csv')
    combined_features_path = os.path.join('Datasets', 'muestra_combined_features.npz')

    # Cargar las bases de datos
    movies = pd.read_csv(movies_path)
    combined_features = np.load(combined_features_path)['arr_0']
    
    # Encuentra el índice de la película en el DataFrame
    try:
        idx = movies[movies['title'].str.lower() == titulo.lower()].index[0]
    except IndexError:
        return [{"error": f"La película '{titulo}' no se encuentra en el dataset."}]

    # Calcular la similitud de coseno entre la película seleccionada y todas las demás
    similitudes = cosine_similarity(combined_features[idx].reshape(1, -1), combined_features)

    # Ordenar las películas por similitud (exceptuando la película actual)
    indices_similares = similitudes[0].argsort()[::-1][1:6]  # Toma las 5 más similares, excluyendo la misma

    # Generar la lista de recomendaciones
    recomendaciones = [
        {
            "titulo": movies.iloc[i]['title'],
            "similitud": float(similitudes[0][i])
        }
        for i in indices_similares
    ]

    return recomendaciones


In [17]:
recomendacion('American Wedding')

[{'titulo': 'Meet the Parents', 'similitud': 0.9999942393515939},
 {'titulo': 'Meet the Fockers', 'similitud': 0.9989368653504107},
 {'titulo': 'American Pie', 'similitud': 0.9988592742766998},
 {'titulo': 'Grumpier Old Men', 'similitud': 0.998762763998953},
 {'titulo': 'The Prince & Me', 'similitud': 0.9986698135857679}]

In [ ]:
# Escalar todas las características
scaler = StandardScaler()
movies_features_scaled = scaler.fit_transform(movies_features)

# Multiplicar la columna `collection_encoded` para ajustar su peso
movies_features_scaled[:, -1] *= 1000  # `collection_encoded` es la última columna